# 并行与并发
#### 并行：即真正的同时运行，如4 cores cpu最大并行数为4.
#### 并发：广义的同时处理，如单核cpu可能在1s内运行了100个进程，从多任务执行角度来说，并发的范围大于并行。
# asyncio与多线程对比
## 多线程

In [1]:
import threading
import itertools
import time
import sys

class Signal:
    go = True

# 任务
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 退格使得光标位置归零
        time.sleep(.1)
        if not signal.go: # 通过可变的signal对象终止spin
            break
    
    write(' ' * len(status) + '\x08' * len(status))        
        
def slow_function():
    time.sleep(3)
    return 42

# 调度
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal)) # 启动spin线程
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function() # 主线程阻塞
    signal.go = False # 向任务线程传递终止信号
    spinner.join() # 主线程阻塞（等待spinner结束）
    
    return result

def main():
    result = supervisor()
    
main()

spinner object: <Thread(Thread-6, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking          

## 异步io 
以下代码报错原因：IPython 内核本身在事件循环上运行，而 asyncio 不允许嵌套其事件​​循环

In [2]:
import asyncio
import itertools
import sys

# 任务协程
@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1) # yield from调用异步等待方法
        except asyncio.CancelledError: # 捕获子生成器处理不了的的asyncio.CancelledError（yield from只传递异常，子生成器若无法处理则抛出该异常）
            break
    
    write(' ' * len(status) + '\x08' * len(status)) 

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42

# 调度协程 
@asyncio.coroutine
def supervisor():
    start = time.time()
    # 异步协程，直接非阻塞继续运行，直接注册slow_function()中的asyncio.sleep(3)。
    spinner = asyncio.async(spin('thinking!')) # 对异步协程排期并开始运行，返回Task对象（立刻返回），Task的创建也可以通过loop.create_task()
    print('spinner object:', spinner)
    result = yield from asyncio.async(slow_function()) # 直接驱动异步协程，协程会挂起直到获取结果（注意不能使用线程阻塞,否则spin协程无法运行）3s
    #     结束一个内部含有未结束协程的协程
    #     spinner.cancel() # 通过任务协程对象（相当于委派生成器）将asyncio.CancelledError传给子生成器asyncio.sleep()
    print("time consumed:", time.time() - start)
    return result

def main():
    loop = asyncio.get_event_loop() # 获取时间循环对象
    result = loop.run_until_complete(supervisor()) # 时间循环开始驱动supervisor()异步协程对象，主线程阻塞直到运行完毕。
    loop.close()
    print('Answer:', result)
    
main()

RuntimeError: This event loop is already running

spinner object: <Task pending coro=<spin() running at <ipython-input-2-b95b39e84091>:6>>
| thinking/ thinking

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: asyncio.async() function is deprecated, use ensure_future()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: asyncio.async() function is deprecated, use ensure_future()


- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinkingtime consumed: 3.008270025253296
- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking!

## 多线程与异步IO
1.asyncio.Task与threading.Thread对象等效。不过Task不能自己手动创建。  
2.获取到Task即已经对异步协程任务进行了排期，而Thread实例必须调用start来启动。  
3.当前协程的休眠需要利用yield from驱动asyncio.sleep(3)方法。  
4.没有API能从外部终止线程，但Task可调用cancel()取消。  
5.异步io中的调度程序supervisor（异步协程）需要loop.run_until_complete(supervisor())执行。
# asyncio.Future
asyncio.Future用于包装协程，__当使用BaseEventLoop.create_task()或asyncio.async创建一个Task（Future子类），实际上对协程进行了排期与包装，这与Executor.submit()创建concurrent.futures.Future实例的理念相同__。不过asyncio.Future的result方法不会阻塞等待，如果协程没有结束则抛出异常。

如果想从asyncio.Future获取结果，__一般使用yield from，该步会将控制权交还给事件循环（相当于协程在当前行挂起）__，__asyncio.Future相当于子生成器，当yield from在yield子生成器生产的结果后控制权移交给事件循环（挂起），当Future中的协程执行完毕，事件循环重新激活委派生成器（yield from所在协程，可能事件循环send(None)吧）与子生成器（asyncio.Future对象）并获取result__。

## 从Future、任务、协程中产出
asyncio包中，可以使用yield from从Future/Task或协程对象中产出结果，__Future/Task和协程对象都是awaitable的对象__，如果需要对协程排期，可以使用aysncio.aysnc()或loop.create_task()方法。
# 使用asyncio和aiohttp包下载

In [5]:
import asyncio
import aiohttp
from flags import BASE_URL, save_flag, show, main

@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower)
    resp = yield from aiohttp.request('GET', url) # 职责委托给子生成器（异步协程）协程在此挂起等待结果
    image = yield from resp.read() # 职责委托给子生成器（异步协程）协程在此挂起等待结果
    return image

@asyncio.coroutine
def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list): # 调度
    loop = asyncio.get_event_loop()
    to_do = [download_one(cc) for cc in sorted(cc_list)]
    wait_coro = asyncio.wait(to_do) # 批量对协程对象进行排期（封装为Task对象），得到一个委派生成器或协程对象
    res, _ = loop.run_until_complete(wait_coro) # 会对wait_coro再次包装成Task，yield from驱动其执行完毕，然后获取wait_coro的结果
    loop.close()
    
    return len(res)

main(download_many)

RuntimeError: This event loop is already running

EG PH BD IR ID JP PK RU IN ET DE CN NG CD TR US MX BR VN FR 

# 协程与异步的关系
为何异步程序需要使用协程（yield from），以yield from驱动呢，当对一个协程（最外层委派生成器）进行排期时，无论其内部使用了多少层委派生成器，每层委派生成器或子生成器（asyncio中的API）是否被包装为Task或Future，最核心的其实是事件循环为其中的所有子生成器（asyncio中的API）进行了异步处理（依赖操作系统）与排期管理，当子生成器（asyncio中的API）执行结束，事件循环将控制权与结果移交至对应行，然后继续执行。当然，如果被排期的协程中使用了协程异步排期，则直接非阻塞继续运行。